In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten

In [2]:
# Load the dataset
file_name = "1_GP.xlsx"
df = pd.read_excel(file_name)

# Preprocess the data
X = df[['Precipitation', 'Airtemp']]  # Features
y = df['swe']  # Target

# Handle missing values
imputer_X = SimpleImputer(strategy='mean')
X_imputed = imputer_X.fit_transform(X)

imputer_y = SimpleImputer(strategy='mean')
y_imputed = imputer_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_imputed, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
# Reshape the data for CNN (assuming time series data)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Define the model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
cnn_history = cnn_model.fit(X_train_reshaped, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=0)

# Evaluate the model
cnn_loss = cnn_model.evaluate(X_test_reshaped, y_test)
print("Convolutional Neural Network (CNN) MSE:", cnn_loss)

NameError: name 'Sequential' is not defined